# GLM Between Analyis 

This script performs within analysis including:
- SM win 3,2,1 0 minus SC win 3,2,1, 0
- OM win 3,2,1 0 minus OC win 3,2,1, 0

## Imports

In [2]:
 # Standard Library
import os
import sys
import random
import warnings
import statistics
from copy import deepcopy

# NumPy, SciPy, Pandas
import numpy as np
import scipy.io
from scipy import stats
from scipy.stats import norm, zscore, pearsonr, sem
from scipy.signal import gaussian, convolve
from scipy.spatial.distance import squareform
import pandas as pd

# Matplotlib & Seaborn
import matplotlib.pyplot as plt
from matplotlib import patches
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set(style='white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

# Nibabel
import nibabel as nib

# Nilearn
from nilearn import image, masking, datasets, plotting, connectome, input_data
from nilearn.input_data import (
    NiftiMasker, MultiNiftiMasker, NiftiLabelsMasker, NiftiSpheresMasker
)
from nilearn.masking import compute_epi_mask, apply_mask
from nilearn.image import (
    concat_imgs, resample_img, mean_img, index_img, resample_to_img, math_img
)
from nilearn.plotting import plot_roi, plot_glass_brain, view_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix

# Brainiak
from brainiak import image as brainiak_image, io as brainiak_io
import brainiak.eventseg.event

# Scikit-learn
from sklearn import preprocessing, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    PredefinedSplit, LeaveOneOut, KFold, GroupShuffleSplit,
    LeavePGroupsOut, LeaveOneGroupOut, train_test_split
)
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import (
    SelectKBest, chi2, f_classif, VarianceThreshold
)
from sklearn.datasets import load_digits
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# DeepDish
import deepdish as dd

# Statsmodels
import statsmodels.stats.multitest as st


In [3]:
from utils import label_lists, find_cond_index, load_epi_data, load_roi_mask,intersect_mask

# Custom Study Functions 

In [5]:
def find_indices(sub, behav_path):
    """
    Load subject behavioral CSV and extract condition labels.
    
    Args:
        sub (str): Subject identifier.
        behav_path (str): Path to the behavioral data directory.
    
    Returns:
        dict: Dictionary of condition names mapping to run indices.
    """
    behav = pd.read_csv(os.path.join(behav_path, f'{sub}_behav_cleaned.csv'))
    labels = behav.iloc[:, 1]
    _, run_order = label_lists(labels, 200)  # Assuming label_lists returns (labels, order)
    return find_condition_indices(run_order)

def organize_bold_data(bold_unsorted, run_indices, cond_a, cond_b):
    """
    Reorder BOLD data based on condition-specific run indices.
    
    Args:
        bold_unsorted (list): List of unsorted BOLD data arrays.
        run_indices (dict): Mapping of condition labels to run indices.
        cond_a (str): First condition label.
        cond_b (str): Second condition label.
    
    Returns:
        tuple: Arrays for cond_a and cond_b, each with two runs.
    """
    a_runs = [bold_unsorted[run_indices[cond_a][0]], bold_unsorted[run_indices[cond_a][1]]]
    b_runs = [bold_unsorted[run_indices[cond_b][0]], bold_unsorted[run_indices[cond_b][1]]]
    print(f"Returning runs for conditions: {cond_a}, {cond_b}")
    return np.asarray(a_runs), np.asarray(b_runs)

def find_condition_indices(run_labels):
    """
    Find run indices for each condition label.
    
    Args:
        run_labels (list): Ordered run condition labels.
    
    Returns:
        dict: Mapping from condition to list of run indices.
    """
    conditions = {'SM': [], 'SC': [], 'OM': [], 'OC': [], 'RE': [0, 9]}
    
    for idx, label in enumerate(run_labels):
        if label in conditions:
            conditions[label].append(idx)
    
    return conditions

def load_confounds(cond_list, sub_list, behav_path, confound_path):
    """
    Load confound time series for each subject and condition.

    Returns:
        dict: conf_sub[sub][cond][run]
    """
    conf_data = {}

    for sub in sub_list:
        cond_data = {}
        run_indices = find_indices(sub, behav_path)

        for cond in cond_list:
            runs = []
            for run_idx in run_indices[cond]:
                fpath = os.path.join(
                    confound_path, sub, "func",
                    f"{sub}_ses-01_task-Attn_run-{run_idx}_desc-model_timeseries.csv"
                )
                data = pd.read_csv(fpath).iloc[4:].values
                runs.append(data)
            cond_data[cond] = runs
        
        conf_data[sub] = cond_data

    return conf_data

def load_two_condition_runs(sub, behav_path, cond_a, cond_b, run_dict, suffix="_205_noproc.npy"):
    """
    Load and organize BOLD data for two conditions (two runs each).
    
    Returns:
        list: Two lists of 2D arrays, one for each condition.
    """
    cats = list(np.load(load_frmi + sub + suffix, allow_pickle=True))
    run_indices = find_indices(sub, behav_path)
    a_runs, b_runs = organize_bold_data(cats, run_indices, cond_a, cond_b)
    return list(a_runs[run_dict[sub][cond_a]]), list(b_runs[run_dict[sub][cond_b]])

def create_event_list(sub, bpress, cond, run_dict, base_onset, comp_onsets, stim_dur):
    """
    Generate event onsets for each run of a subject.
    
    Returns:
        dict: run-indexed event DataFrames.
        list: window labels used.
    """
    all_onsets = list(np.asarray(bpress[sub][cond])[run_dict[sub][cond]])
    events_dict = {}
    window_labels = []

    for run_idx, onset_array in enumerate(all_onsets):
        onset_array = onset_array.astype(float)
        trial_types = ['win0'] * len(onset_array)
        durations = [stim_dur] * len(onset_array)
        onsets = [t + base_onset for t in onset_array]

        for i, comp_onset in enumerate(comp_onsets):
            label = f'win{i+1}'
            trial_types += [label] * len(onset_array)
            durations += [stim_dur] * len(onset_array)
            onsets += [t + comp_onset for t in onset_array]
            window_labels.append(label)

        events_df = pd.DataFrame({
            'onset': onsets,
            'trial_type': trial_types,
            'duration': durations
        })
        events_dict[run_idx] = events_df

    return events_dict, window_labels



# Directories

In [1]:
data_dir = '../../data/bids' # where the fMRI data is downloaded
rois_dir = ... # not specified 
behav_p = '../behavioral/stim_list'
load_bpress = "../behavioral/"
load_fmri = '...' #location of preprocessed data output from preproc scripts
out_dir = "../../data/output" # where to save the directory
confounds_dir = '../behavioral_data/confound_info/'

# Sublist

In [1]:
sub_list = ["sub-000","sub-001","sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018", "sub-019", "sub-020","sub-021",'sub-022','sub-023','sub-024','sub-025','sub-026','sub-027']


# Analysis variables

In [21]:
# ** Only need to change inputs below!
hm_thresh = str(3) ## framewise displacement threshold 
prefix = 'between'
sav_dir = 'n28_p'+hm_thresh+'_betas_4fhwm_hp001p25_shaefGM_excOvlp'

# start of zero-th window in seconds
base_onset = -6
# how many windows and what is the onset start time
# we're now centering on the TR in which the bpress occured - onset of win 1
# -3-3,-3+3,-3+6
comp_onset_list = [-3,0,3]
stim_dur = 3 # how long is the stimuous on

# set the conditions conditions #
cond_all = np.asarray([['SM','SC'],['OM', 'OC']])
# In seconds what is the lag between a stimulus onset and the peak bold response
sec_lag = 0

# Create dir
path = out_dir+sav_dir
try: 
    os.mkdir(os.path.join(path))
except OSError as error: 
    print(error)  

[Errno 17] File exists: '/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/n28_p3_zscr_4fhwm_hp001p25_shaefGM_excOvlp'


# Static variables

In [12]:
tr = 1.5  # repetition time is 1 second
n_scans = 205 # the acquisition comprises 204 scans
frame_times = np.arange(n_scans) * tr  # here are the correspoding frame times given TRs
# subject dics #
group_sub_glm_a = {}
group_sub_glm_b = {}
# sublist for individ analysis below # 
excl_sub_list = []

In [13]:
## LOAD MASK # 
# mask in study was derived f rom schaeffer parcellation. To find analagous grey matter mask, see: https://brainiak.org/tutorials/01-setup/
# alternatively, see the script 'generate_masks'
gm_shaef = nib.load('../../data/shaef_gm_MNI_mask.nii')


In [15]:
## GET BUTTON PRESS INFO PER SUBJECT  ##
bpress = dict(enumerate(np.load(os.path.join(load_bpress, "n28_4_conds_ts_press_ovrlpREMOV.npy"), 
                                allow_pickle=True).flatten(),1))[1]

In [16]:
### runs to exclude with head motion accounted for and missing bpress runs deleted
run_dic = dict(enumerate(np.load(os.path.join(confounds_dir, "n28_runs_2_include_removNoBpress_delHMruns_threshp%s.npy") %(hm_thresh), 
                                allow_pickle=True).flatten(),1))[1]


In [ ]:
## load confounds 
conf_sub = dict(enumerate(np.load(os.path.join(confounds_dir, 'n28_conf+cens_MERGE_removNoBpress_delHMruns_threshp%s_glm.npy')%(hm_thresh), 
                                          allow_pickle = True).flatten(),1))[1]
    

# GLM variables 

In [18]:
fmri_glm = FirstLevelModel(t_r=1.5,
                           signal_scaling=False,
                           hrf_model = 'glover',
                           drift_order=None,
                           mask_img = gm_shaef,
                           high_pass=None,
                           drift_model=None,
                           smoothing_fwhm=4,
                           standardize=True,
                           minimize_memory=False)

In [19]:
fmri_glm.get_params()

{'drift_model': None,
 'drift_order': None,
 'fir_delays': [0],
 'high_pass': None,
 'hrf_model': 'glover',
 'mask_img': <nibabel.nifti1.Nifti1Image at 0x7f0970c733d0>,
 'memory': Memory(location=None),
 'memory_level': 1,
 'min_onset': -24,
 'minimize_memory': False,
 'n_jobs': 1,
 'noise_model': 'ar1',
 'signal_scaling': False,
 'slice_time_ref': 0.0,
 'smoothing_fwhm': 4,
 'standardize': True,
 'subject_label': None,
 't_r': 1.5,
 'target_affine': None,
 'target_shape': None,
 'verbose': 0}

# Run

In [ ]:
for cond_list in cond_all:
    cond_a = cond_list[0]
    cond_b = cond_list[1]
    print('cur conds:', cond_list, 'at HM thresh', hm_thresh)
    
    for sub in sub_list:
        # skip sub if both runs for a condition are unavailable (see head_mot)
        if conf_sub[sub][cond_a] == [] or conf_sub[sub][cond_b] == []: continue
        excl_sub_list.append(sub)
        print('design mat for', sub)
        
        sub_conts_a = {}
        sub_conts_b = {}
        # Create events for all four runs
        events_a, window_list = create_event_list(sub, bpress, cond_a,run_dic, base_onset,comp_onset_list,stim_dur)
       
        # If rest condition, then use button presses from condition A 
        if cond_b == 'RE': 
            events_b,window_list = create_event_list(sub, bpress, cond_a,run_dic,base_onset,comp_onset_list,stim_dur)
        else:
            events_b,window_list = create_event_list(sub, bpress, cond_b,run_dic,base_onset,comp_onset_list,stim_dur)
            
        # Create design matrix - SM
        design_matrices_a = [make_first_level_design_matrix(frame_times, events_a[df], hrf_model = 'glover',
                      drift_model=None, add_regs=conf_sub[sub][cond_a][df]) for df in events_a]
        # Create design matrix - SC (df +1 cuz regs are index 1,2, not 0, 1)
        design_matrices_b = [make_first_level_design_matrix(frame_times, events_b[df], hrf_model = 'glover',
                      drift_model=None, add_regs=conf_sub[sub][cond_b][df]) for df in events_b]

        # FMRI - high pass, low pass filter
        cond_a_runs_temp, cond_b_runs_temp = load_2conds_runs_fmri(sub,behav_p,cond_a,cond_b,run_dic)
        cond_a_runs = [nil.image.clean_img(unclean_img, detrend=False, 
                                        standardize=False, 
                                        low_pass=.25,high_pass=.001,t_r=1.5) for unclean_img in cond_a_runs_temp]
        cond_b_runs = [nil.image.clean_img(unclean_img, detrend=False, 
                                        standardize=False, 
                                        low_pass=.25,high_pass=.001,t_r=1.5) for unclean_img in cond_b_runs_temp]
        

        # FIT GLM per subject 
        print('fit glm')
        window_list = ['win0','win1','win2','win3']
        for win in window_list:
            # group sub glm is now a nested dictionary with three contrasts
            ## COND A ## 
            fmri_glm = fmri_glm.fit(cond_a_runs, design_matrices=design_matrices_a) 
            # Compute three contrasts #
            sub_conts_a[win] = fmri_glm.compute_contrast(
                    win, output_type='effect_size') #effect_size
            # each window saved as a key
            group_sub_glm_a[sub] = sub_conts_a
            ## Save ## 
            np.save(os.path.join(out_dir+sav_dir, '%s_%s.npy') %(cond_a, prefix), group_sub_glm_a)

            ## COND B ## 
            fmri_glm = fmri_glm.fit(cond_b_runs, design_matrices=design_matrices_b) 
            # Compute three contrasts #
            sub_conts_b[win] = fmri_glm.compute_contrast(
                    win, output_type='effect_size') #effect_size

            group_sub_glm_b[sub] = sub_conts_b
            ## Save ## 
            np.save(os.path.join(out_dir+sav_dir, '%s_%s.npy') %(cond_b,prefix), group_sub_glm_b)


            # subtract images
            full_contrast_win = math_img("img1 - img2",img1=group_sub_glm_a[sub][win],img2=group_sub_glm_b[sub][win])

            # save individual full contrast subtracted images for t-test
            nib.save(full_contrast_win, os.path.join(
                out_dir+sav_dir,'%s_%s_%s_%s_zmap.nii') % (sub,cond_a+'-'+cond_b, prefix, win))
            
            print('finish',win)
        print('finish', sub)


cur conds: ['OM' 'OC'] at HM thresh 3
design mat for sub-015
returning OM OC
fit glm
finish win0
finish win1
finish win2
finish win3
finish sub-015
